In [ ]:
require 'numo/narray'
require 'numo/gsl'
require 'numo/gnuplot'
require 'ostruct'
require 'daru'
require 'fileutils'
require 'pry'
require 'spreadsheet'

In [ ]:
IRuby::Kernel.instance.switch_backend!(:pry)

### Utilities

In [ ]:
# http://youinfinitesnake.blogspot.com/2011/02/attractive-scientific-plots-with.html
# http://www.gnuplotting.org/attractive-plots/

def plotify(filename)
  Numo.noteplot do |note|
    set terminal: 'pdfcairo font "Gill Sans,9" size 29cm, 21cm linewidth 2 rounded fontscale 0.75'
    set output: filename
    
    set style: 'line 80 lt rgb "#808080"'
    set style: 'line 81 lt 0'
    
    set lmargin: 15
    set rmargin: 15
    set tmargin: 5
    set bmargin: 5

    # Remove border on top and right.  These
    # borders are useless and make it harder
    # to see plotted lines near the border.
    # Also, put it in grey; no need for so much emphasis on a border.
    set border: '3 back linestyle 80'
    set grid: 'back linestyle 81'
    set xtics: 'nomirror'
    set ytics: 'nomirror'
    
    set key: 'bottom right'
    
    # Line styles: try to pick pleasing colors, rather
    # than strictly primary colors or hard-to-see colors
    # like gnuplot's default yellow.  Make the lines thick
    # so they're easy to see in small plots in papers.
    set style: 'line 1 lt rgb "#A00000" lw 1 pt 1'
    set style: 'line 2 lt rgb "#00A000" lw 1 pt 6'
    set style: 'line 3 lt rgb "#5060D0" lw 1 pt 2'
    set style: 'line 4 lt rgb "#F25900" lw 1 pt 9'
    
    yield note
  end
end

### Data analysis

In [ ]:
# Original data
df = Daru::DataFrame.from_csv('work/labs/lab-1.2/data/data.csv')

# Error estimates
delta_theta = Math::PI / 180.0

left, right = df['left'], df['right']
# Estimate mean and error
df['n'] = (left + right) / 2.0
df['delta(n)'] = (right - left).abs / 2.0
df['1 - cos'] = df['theta'].map { |e| 1 - Math.cos(e * Math::PI / 180.0) }
df['1/n'] = df['n'].map { |e| 1/e }
df['1/n - 1/n0'] = df['1/n'] - df['1/n'][0]
df['delta(1/n)'] = df['1/n'] ** 2.0 * df['delta(n)']
df['delta(1/n - 1/n0)'] = (df['delta(1/n)'] ** 2.0 + df['delta(1/n)'][0] ** 2.0).sqrt
df['delta(1 - cos(theta))'] = df['theta'].map { |e| Math.sin(e * Math::PI / 180.0) * delta_theta }

x = df['1 - cos']
y = df['1/n - 1/n0']
x_err = df['delta(1 - cos(theta))']
y_err = df['delta(1/n - 1/n0)']

# Number of points to sample
n_samples = 1000

nx = Numo::DFloat[x.to_a]
ny = Numo::DFloat[y.to_a]
nw = Numo::DFloat[y_err.to_a]
best_fit = Numo::GSL::Fit.linear(nx, ny)
xrange = Numo::DFloat.linspace(nx[0], nx[-1], n_samples)
best_fit_y, best_fit_err = *Numo::GSL::Fit.linear_est(xrange, best_fit)

c0 = best_fit.c0[0]
c1 = best_fit.c1[0]

c0_err = Math.sqrt best_fit.cov00[0]
c1_err = Math.sqrt best_fit.cov11[0]

puts ("c0 = %1.3e +- %1.3e" % [c0, c0_err])
puts ("c1 = %1.3e +- %1.3e" % [c1, c1_err])

In [ ]:
# Gamma energy
e_gamma = 662 # KiloElectronvolts
one_over_n0 = df['1/n'][0]

get_n_for_theta = -> (theta) do
  one_over_n = one_over_n0 + c0 + c1 * (1 - Math.cos(theta * Math::PI / 180.0))
  1 / one_over_n
end

get_delta_n_for_theta = -> (theta) do
  angle_in_radians = theta * Math::PI / 180.0

  c0_cum_err_sq = c0_err ** 2.0
  one_over_n0_cum_err_sq = df['delta(1/n)'][0] ** 2.0
  c1_cum_err_sq = ((1 - Math.cos(angle_in_radians)) * c1_err) ** 2.0
  angle_cum_err_sq = (c1 * Math.sin(angle_in_radians) * delta_theta) ** 2.0

  puts "c0_cum_err_sq: %1.3e" % [c0_cum_err_sq]
  puts "one_over_n0_cum_err_sq: %1.3e" % [one_over_n0_cum_err_sq]
  puts "c1_cum_err_sq: %1.3e" % [c1_cum_err_sq]
  puts "angle_cum_err_sq: %1.3e" % [angle_cum_err_sq]

  delta = Math.sqrt(c0_cum_err_sq +
          one_over_n0_cum_err_sq +
          c1_cum_err_sq +
          angle_cum_err_sq)
  denom = (c0 + one_over_n0 + c1 * (1 - Math.cos(angle_in_radians))) ** 2.0
  delta / denom
end

puts "n0 = %.3f" % [n0 = get_n_for_theta.call(0)]
puts "n90 = %.3f" % [n90 = get_n_for_theta.call(90)]
puts "delta_n0 = %.2f" % [delta_n0 = get_delta_n_for_theta.call(0)]
puts "delta_n90 = %.2f" % [delta_n90 = get_delta_n_for_theta.call(90)]

e = n90 / (n0 - n90) * e_gamma
delta_e = e_gamma * Math.sqrt(
  ( delta_n90 * n0 / ((n0 - n90) ** 2.0) ) ** 2.0 +
  ( delta_n0 * n90 / ((n0 - n90) ** 2.0) ) ** 2.0
)

puts "e = %.2f" % [e]
puts "delta_e = %.2f" % [delta_e]

### Plots

In [ ]:
plotify 'work/labs/lab-1.2/output/plot1.pdf' do |note|
  note.set xlabel: '1 - cos(t)'
  note.set ylabel: '1/n - 1/n_0, 1/n_{channel}'
  note.set title: 'Рис. 1 Зависимость 1/n от t'
  note.set format: { y:'%2.0e' }
  note.set style: 'fill transparent solid 0.5' # partial transparency
  
  note.plot [x, y, x_err, y_err, {ls:1, with: 'xyerrorbars', title: 'Experimental data: 1/n - 1/n0'}],
            [xrange, best_fit_y, {ls:3, with: 'lines', title: 'Best fit: c0 + c1 * (1 - cos(t))'}],
            [xrange, best_fit_y + best_fit_err, best_fit_y - best_fit_err,
                {ls:3, with: 'filledcurves', title: 'Best fit error margin'}]
end

### Saving intermediate data

In [ ]:
df.write_excel('work/labs/lab-1.2/output/table1.xlsx')